In [1]:
!pip install torch torchaudio transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import soundfile as sf
import torchaudio

In [3]:
# Load the pre-trained Wav2Vec model
# processor = Wav2Vec2Processor.from_pretrained("othrif/wav2vec2-large-xlsr-arabic")
# config = Wav2Vec2Config.from_pretrained("othrif/wav2vec2-large-xlsr-arabic", output_hidden_states=True) # to get the hidden state
# wav2vec_model = Wav2Vec2ForCTC.from_pretrained("othrif/wav2vec2-large-xlsr-arabic", config=config)
# processor = Wav2Vec2Processor.from_pretrained("FarisHijazi/wav2vec2-large-xls-r-300m-arabic-colab")
# wav2vec_model = Wav2Vec2ForCTC.from_pretrained("FarisHijazi/wav2vec2-large-xls-r-300m-arabic-colab")
processor = Wav2Vec2Processor.from_pretrained("othrif/wav2vec2-large-xlsr-arabic")
# wav2vec_model = Wav2Vec2ForCTC.from_pretrained("othrif/wav2vec2-large-xlsr-arabic")
wav2vec_model = Wav2Vec2Model.from_pretrained("othrif/wav2vec2-large-xlsr-arabic")

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at othrif/wav2vec2-large-xlsr-arabic and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Load Common Voice dataset
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="train+validation+test")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9010it [00:00, 90090.85it/s]
Reading metadata...: 28167it [00:00, 111665.06it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10409it [00:00, 115683.86it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10445it [00:00, 97266.48it/s] 


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 12626it [00:00, 126249.28it/s]
Reading metadata...: 36192it [00:00, 136663.16it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14994it [00:00, 131471.50it/s]


In [6]:
print(dataset)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 49021
})


In [7]:
# Removing unwanted features (we only want audio and its transcription)
dataset = dataset.remove_columns(["client_id", "path", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 49021
})


In [8]:
print(dataset[0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([-2.13162821e-14, -1.01252340e-13, -3.55271368e-14, ...,
        1.26173454e-08,  1.54317320e-07,  1.32340631e-07]), 'sampling_rate': 48000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [9]:
class CommonVoiceDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load and resample audio
        audio_input, sampling_rate = torchaudio.load(self.dataset[idx]["audio"]["path"])
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio_input = resampler(audio_input)

        # Process audio
        # input_values = self.processor(audio_input.squeeze(), sampling_rate=16000, return_tensors="pt").input_values.squeeze()
        input_values = self.processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values
        input_values = input_values.squeeze(0)
        input_length = input_values.shape[1]
        # print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
        # print(len(input_values))
        # print(input_values)
        # print(input_values.shape)
        # print(input_length)
        # print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")

        # Process labels
        # print("bbbbbbbbbbbbbbbbbbbbbbbbbbbb")
        labels = self.processor.tokenizer.encode(self.dataset[idx]["sentence"])
        labels = torch.tensor(labels)
        # print(labels)
        # print("eeeeeeeeeeeeeeeeeeeeeeeeeeee")
        labels = torch.tensor(labels)



        return input_values, labels, input_length

In [10]:
# Instantiate the dataset
common_voice_dataset = CommonVoiceDataset(dataset, processor)

In [11]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [12]:
import torch.nn.functional as F

In [13]:
class CustomSTTModel(nn.Module):
    def __init__(self, wav2vec_model, lstm_hidden_size, lstm_layers, attention_heads):
        super(CustomSTTModel, self).__init__()
        self.wav2vec = wav2vec_model

        # Freeze Wav2Vec model parameters
        for param in self.wav2vec.parameters():
            param.requires_grad = False

        # Get the feature size from Wav2Vec model
        feature_size = self.wav2vec.config.hidden_size

        self.lstm = nn.LSTM(input_size=feature_size,
                            hidden_size=lstm_hidden_size,
                            num_layers=lstm_layers,
                            batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=lstm_hidden_size,
                                               num_heads=attention_heads,
                                               batch_first=True)
        self.output_layer = nn.Linear(lstm_hidden_size, wav2vec_model.config.vocab_size)

    def forward(self, input_values, input_lengths):

      self.wav2vec.eval()
      with torch.no_grad():
          # print("input_values: ", len(input_values[0]))
          # print("input_values: ", input_values[0])
          # print("input_values: ", input_values[0].shape)
          # print("input_lengths: ", input_lengths)
          # print("vocab size: ", self.wav2vec.config.vocab_size)
          wav2vec_output = self.wav2vec(input_values).last_hidden_state

      processed_lengths = wav2vec_output.shape[1]
      processed_lengths = torch.full((wav2vec_output.shape[0],), processed_lengths, dtype=torch.int64)
      # print("processed_lengths: ", processed_lengths)
      sorted_lengths, sorted_indices = input_lengths.sort(descending=True)
      sorted_wav2vec_output = wav2vec_output[sorted_indices]

      # Debugging
      # print("feature size: ", self.wav2vec.config.hidden_size)
      # print("Sorted lengths:", sorted_lengths)
      # print("Shape of wav2vec_output:", wav2vec_output.shape)
      # print("Shape of sorted_wav2vec_output:", sorted_wav2vec_output.shape)

      packed_input = pack_padded_sequence(sorted_wav2vec_output, processed_lengths.cpu(), batch_first=True)
      # print("packed_input shape", packed_input)
      if input_values.is_cuda:
          self.lstm.flatten_parameters()

      packed_lstm_output, _ = self.lstm(packed_input)
      # print('yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy')
      lstm_output, _ = pad_packed_sequence(packed_lstm_output, batch_first=True)
      # print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz")

      # Attention mechanism
      attention_output, _ = self.attention(lstm_output, lstm_output, lstm_output)

      # Fully connected layer
      output = self.output_layer(attention_output)

      # Apply softmax to convert logits to probabilities
      # output = F.log_softmax(output, dim=2)

      # print("sorted_lengths:", sorted_lengths)
      # print("sorted_wav2vec_output:", sorted_wav2vec_output)
      # print("packed_input:", packed_input)
      # print("packed_lstm_output:", packed_lstm_output)
      # print("lstm_output:", lstm_output)
      # print("attention_output:", attention_output)
      # print("output:", output)
      return output


In [14]:
# Instantiate the model
custom_model = CustomSTTModel(wav2vec_model, lstm_hidden_size=128, lstm_layers=2, attention_heads=4)

In [15]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    # Sort the batch by decreasing order of sequence length
    batch = sorted(batch, key=lambda x: x[2], reverse=True)

    input_values, labels, input_lengths = zip(*batch)
    # print("aaaaaaaaaaaaaaaaaaaaaaaaa")
    # print(input_values)
    # print(labels)
    # print(input_lengths)
    # print("bbbbbbbbbbbbbbbbbbbbbbb")

    # Pad the input_values. Ensure each tensor in input_values is 1D
    input_values_padded = pad_sequence([iv.squeeze() for iv in input_values], batch_first=True)

    # Pad the labels
    labels_padded = pad_sequence(labels, batch_first=True)

    # Convert input_lengths to tensor
    input_lengths = torch.tensor([iv.shape[1] for iv in input_values], dtype=torch.long)

    # print("ccccccccccccccccccccccccc")
    # print(input_values_padded)
    # print(labels_padded)
    # print(input_lengths)
    # print("ddddddddddddddddddddddddddd")

    return input_values_padded, labels_padded, input_lengths

In [16]:
# Create a DataLoader
batch_size = 8 # Adjust as needed
train_loader = DataLoader(common_voice_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [37]:
best_loss = 10000

def train(model, data_loader, criterion, optimizer, epochs, device):
    global best_loss
    model.to(device)

    for epoch in range(epochs):
        model.train()
        batch_idx = 0
        total_loss = 0
        for batch in data_loader:
            # Unpack the batch and move to the appropriate device
            input_values, labels, input_lengths = batch
            input_values, labels = input_values.to(device), labels.to(device)

            # Forward pass
            output = model(input_values, input_lengths.cpu())  # Ensure lengths is on CPU
            # print("output:", output.shape)
            # print("labels:", labels.shape)
            output_lengths = output.shape[1]
            output_lengths = torch.full((output.shape[0],), output_lengths, dtype=torch.int64)
            # print("output_lengths", output_lengths)
            # print("input_lengths", input_lengths)
            # print("mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm")
            log_probs = torch.nn.functional.log_softmax(output, dim=2) # CTC expects log softmax probabilities
            log_probs = log_probs.permute(1, 0, 2)
            # Compute loss
            label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long, device=device)

            # Print intermediate information for debugging
            # print("Output shape:", output.shape)
            # print("Labels:", labels)
            # print("Label lengths:", label_lengths)
            # print(output_lengths)
            # print(log_probs.shape)
            # print(label_lengths)
            # print(labels.shape)

            loss = criterion(log_probs, labels, output_lengths, label_lengths)

            # Calculate the total loss per batch
            total_loss += loss.item()

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()

            print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}")
            batch_idx+=1

        # Compute average loss for the epoch
        avg_loss = total_loss / len(data_loader)

        # Save the model every two epochs
        if epoch % 2 == 0:
            torch.save(model.state_dict(), f'model_state_dict_epoch_{epoch}.pth')

        # Update the best model if current model is better
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), 'best_model_state_dict.pth')

In [38]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
from torch import nn
import torch.optim as optim

# Define the Loss Function (Criterion)
# The CTC loss function expects logits as inputs, so ensure your model outputs logits

criterion = nn.CTCLoss(blank=processor.tokenizer.pad_token_id).to(device)

In [40]:
# Define the Optimizer
optimizer = optim.Adam(custom_model.parameters(), lr=0.001)

In [41]:
# Run the training
epochs = 10  # Define the number of epochs
train(custom_model, train_loader, criterion, optimizer, epochs, device)

<ipython-input-9-88ba02b903e0>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


Epoch [1/10], Batch [1/6128], Loss: 1.9198
Epoch [1/10], Batch [2/6128], Loss: 1.7191
Epoch [1/10], Batch [3/6128], Loss: 2.7607
Epoch [1/10], Batch [4/6128], Loss: 2.9049
Epoch [1/10], Batch [5/6128], Loss: 2.7269
Epoch [1/10], Batch [6/6128], Loss: 2.3180
Epoch [1/10], Batch [7/6128], Loss: 2.2378
Epoch [1/10], Batch [8/6128], Loss: 2.2178


KeyboardInterrupt: ignored

In [ ]:
# save state dictionary since model has parameters or layers that are wrapped by utilities
torch.save(custom_model.state_dict(), f'model_state_dict.pth')

In [ ]:
# To load the model state dictionary later
# Recreate the model architecture and load the state dictionary into it
model = CustomSTTModel(wav2vec_model, lstm_hidden_size=128, lstm_layers=2, attention_heads=4)
model.load_state_dict(torch.load('model_state_dict.pth'))
model.eval()  # for inference

In [ ]:
def greedy_decode(outputs, blank_label):
    """
    Decode the output of the model using a greedy approach.
    """
    arg_maxes = torch.argmax(outputs, dim=2)
    print(arg_maxes.shape)
    decodes = []
    for i in range(arg_maxes.size(0)):
        decode = []
        for j in range(arg_maxes.size(1)):
            if arg_maxes[i][j] != blank_label:
                if j != 0 and arg_maxes[i][j-1] == arg_maxes[i][j]:
                    continue
                decode.append(arg_maxes[i][j].item())
        decodes.append(decode)
    return decodes

In [ ]:
from torch.nn.functional import log_softmax

In [ ]:
def predict(model, processor, input_values, input_length, device):
    """
    Generate predictions from the model.
    """
    model.eval()
    input_values = input_values.to(device)

    with torch.no_grad():
        # Forward pass
        logits = model(input_values, input_length)

        # Apply log softmax to get log probabilities
        log_probs = log_softmax(logits, dim=2)

        # Decode the model output
        decoded_preds = greedy_decode(log_probs, blank_label=processor.tokenizer.pad_token_id)

        # Convert predictions to text
        decoded_text = [processor.decode(pred) for pred in decoded_preds]

    return decoded_text

In [ ]:
def process_audio_file(file_path, processor, target_sample_rate=16000):
    # Load audio file
    audio_input, sampling_rate = torchaudio.load(file_path)

    # Convert stereo to mono by averaging the two channels if necessary
    if audio_input.shape[0] == 2:
        audio_input = torch.mean(audio_input, dim=0, keepdim=True)

    # Resample the audio file if necessary
    if sampling_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sample_rate)
        audio_input = resampler(audio_input)

    # Process audio to the format expected by the model
    input_values = processor(audio_input, sampling_rate=target_sample_rate, return_tensors="pt").input_values
    input_values = input_values.squeeze(0)
    input_length = input_values.shape[1]
    input_length = torch.tensor([input_length], dtype=torch.long)
    return input_values, input_length

In [ ]:
# Example usage
file_path = 'r2.wav'
input_values, input_length = process_audio_file(file_path, processor)

In [ ]:
print(input_values.shape)
print(input_length)

torch.Size([1, 57686])
tensor([57686])


In [ ]:
predictions = predict(custom_model, processor, input_values, input_length, device)
print(predictions)

torch.Size([1, 180])
['ٌ']
